# Second Cleanup after first attempt of visualization

Imports

In [61]:
import pandas as pd
import requests
import json
import numpy as np
import re
import unicodedata

Load Data

In [37]:
df = pd.read_csv("/Users/emilywu/Desktop/DataLiteracy/MiniProjekt/clean_shark_attacks.csv")

In [38]:
display(df)
df.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude
0,0,10th January,2026,Unprovoked,Australia,Avalon Beach,Surfing,Paul Stanton,M,?,puncture mark to left thumb,N,Unknown,AU,-33.636504,151.329030
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,-35.008658,138.529591
2,2,3rd January,2026,Unprovoked,New Caledonia,Between Bourail and Moindou,Scuba Diving,Unknown,M,?,Injuries to upper limbs,N,Unknown,NC,-21.567724,165.497130
3,3,21st December,2025,Unprovoked,USA,Lovers Point Pacific Grove,Swimming,Erica Fox,F,55,Taken by shark body recovered with multiple in...,Y,Great White Shark,US,36.623394,-121.906667
4,4,12th December,2025,Unprovoked,USA,Salmon Creek,Surfing,Unknown,M,?,Hand Injury,N,Suspected Great White Shark,US,45.712902,-122.664348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2490,2808,03-Feb-2000,2000,Unprovoked,NEW ZEALAND,Oreti Beach (reported as the 4th person bitten...,Surfing,Michael Petas,M,12,"No injury, wetsuit punctured",N,NaN,NZ,-46.421301,168.219569
2491,2809,01-Feb-2000,2000,Unprovoked,AUSTRALIA,"Point Sinclair, Cactus Beach near Penong",Surfing,Anthony Hayes,M,26,Hand bitten,N,3 m [10'] shark,AU,-31.928309,133.011075
2492,2810,Reported 28-Jan-2000,2000,Watercraft,REUNION,Saint Pierre,Canoe with 3 men onboard sank,Boulabhaï Ishmael,M,NaN,FATAL,Y,NaN,RE,48.383272,7.471873
2493,2811,05-Jan-2000,2000,Unprovoked,THAILAND,Phang nga Island,Diving,Stephan Kahl,M,35,FATAL,Y,NaN,TH,9.334030,98.323241


Unnamed: 0     2495
Date           2495
Year           2495
Type           2495
Country        2495
Location       2495
Activity       2413
Name           2467
Sex            2400
Age            1942
Injury         2488
Fatal Y/N      2482
Species        1586
CountryCode    2495
latitude       2495
longitude      2495
dtype: int64

Check if returned coordinates lay within the country of the given country

In [17]:
'''def get_country_code(lat, lon):
    headers = {"User-Agent": "shark-attack-map/1.0 (contact: e12217684@student.tuwien.ac.at)"}
    params = {"lat": f"{lat}", "lon": f"{lon}", "format": "json"}
    response = requests.get("https://nominatim.openstreetmap.org/reverse",headers=headers, params=params)
    if response.status_code == 200:
        res = response.json()
        print(res)
        return res["address"]["country_code"]
    else:
        return (None, None)'''

In [39]:
def get_country_code(lat, lon, session, cache, sleep_s=1.1):
    if pd.isna(lat) or pd.isna(lon):
        return np.nan

    try:
        lat = float(lat); lon = float(lon)
    except ValueError:
        return np.nan

    # Cache
    key = (round(lat, 5), round(lon, 5))
    if key in cache:
        return cache[key]

    headers = {
        "User-Agent": "shark-attack-map/1.0 (contact: e12217684@student.tuwien.ac.at)"
    }
    params = {"lat": f"{lat}", "lon": f"{lon}", "format": "json"}

    try:
        r = session.get(
            "https://nominatim.openstreetmap.org/reverse",
            headers=headers,
            params=params,
            timeout=15
        )
  

        if r.status_code != 200:
            cache[key] = np.nan
            return np.nan

        res = r.json()
        print(res)
        cc = res.get("address", {}).get("country_code", None)
        cc = cc.upper() if cc else np.nan

        cache[key] = cc
    
        return cc

    except requests.RequestException:
        cache[key] = np.nan
        return np.nan

In [40]:
session = requests.Session()
cache = {}

df["fetched_country_code"] = df.apply(
    lambda row: get_country_code(row["latitude"], row["longitude"], session, cache),
    axis=1
)

{'place_id': 398014859, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 12536208940, 'lat': '-33.6365340', 'lon': '151.3290692', 'class': 'amenity', 'type': 'restaurant', 'place_rank': 30, 'importance': 3.500190737769158e-05, 'addresstype': 'amenity', 'name': 'Alma', 'display_name': 'Alma, 47, Old Barrenjoey Road, Avalon Beach, Northern Beaches, Sydney, Northern Beaches Council, New South Wales, 2107, Australia', 'address': {'amenity': 'Alma', 'house_number': '47', 'road': 'Old Barrenjoey Road', 'suburb': 'Avalon Beach', 'borough': 'Northern Beaches', 'city': 'Sydney', 'municipality': 'Northern Beaches Council', 'state': 'New South Wales', 'ISO3166-2-lvl4': 'AU-NSW', 'postcode': '2107', 'country': 'Australia', 'country_code': 'au'}, 'boundingbox': ['-33.6365840', '-33.6364840', '151.3290192', '151.3291192']}
{'place_id': 17598037, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_ty

In [42]:
display(df)
df.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code
0,0,10th January,2026,Unprovoked,Australia,Avalon Beach,Surfing,Paul Stanton,M,?,puncture mark to left thumb,N,Unknown,AU,-33.636504,151.329030,AU
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,-35.008658,138.529591,AU
2,2,3rd January,2026,Unprovoked,New Caledonia,Between Bourail and Moindou,Scuba Diving,Unknown,M,?,Injuries to upper limbs,N,Unknown,NC,-21.567724,165.497130,FR
3,3,21st December,2025,Unprovoked,USA,Lovers Point Pacific Grove,Swimming,Erica Fox,F,55,Taken by shark body recovered with multiple in...,Y,Great White Shark,US,36.623394,-121.906667,US
4,4,12th December,2025,Unprovoked,USA,Salmon Creek,Surfing,Unknown,M,?,Hand Injury,N,Suspected Great White Shark,US,45.712902,-122.664348,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2490,2808,03-Feb-2000,2000,Unprovoked,NEW ZEALAND,Oreti Beach (reported as the 4th person bitten...,Surfing,Michael Petas,M,12,"No injury, wetsuit punctured",N,NaN,NZ,-46.421301,168.219569,NZ
2491,2809,01-Feb-2000,2000,Unprovoked,AUSTRALIA,"Point Sinclair, Cactus Beach near Penong",Surfing,Anthony Hayes,M,26,Hand bitten,N,3 m [10'] shark,AU,-31.928309,133.011075,AU
2492,2810,Reported 28-Jan-2000,2000,Watercraft,REUNION,Saint Pierre,Canoe with 3 men onboard sank,Boulabhaï Ishmael,M,NaN,FATAL,Y,NaN,RE,48.383272,7.471873,FR
2493,2811,05-Jan-2000,2000,Unprovoked,THAILAND,Phang nga Island,Diving,Stephan Kahl,M,35,FATAL,Y,NaN,TH,9.334030,98.323241,TH


Unnamed: 0              2495
Date                    2495
Year                    2495
Type                    2495
Country                 2495
Location                2495
Activity                2413
Name                    2467
Sex                     2400
Age                     1942
Injury                  2488
Fatal Y/N               2482
Species                 1586
CountryCode             2495
latitude                2495
longitude               2495
fetched_country_code    2488
dtype: int64

Get all rows where no country code could be fetched

In [45]:
df_missing = df[df["fetched_country_code"].isna()]
display(df_missing)

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code
97,102,2024-06-02 00:00:00,2024,Unprovoked,Maldive Islands,Indian Ocean,Military ocean training,Moham Zidhan,M,23,Wounds to lower extremities Fatal,Y,Not specified,MV,-10.000000,70.000000,NaN
543,591,29-Jul-2019,2019,Unprovoked,USA,Molokai Channel/,Swimming,Adherbal Treidler de Oliveira,M,49,Bite to abdomen & left leg medial thigh,N,Cookiecutter shark,US,49.989220,-2.238470,NaN
1669,1881,17-Mar-2009,2009,Unprovoked,USA,Alenuihaha Channel,Swimming,Mike Spaulding,M,61,"Minor injury, bite chest and left calf",N,Thought to involve a cookiecutter shark,US,20.529242,-155.903230,NaN
1746,1967,18-Jul-2008,2008,Unprovoked,EGYPT,Daedalus Reef,Diving,Russian male,M,NaN,Leg severed,N,Oceanic whitetip shark,EG,24.934445,35.868120,NaN
1824,2058,27-Sep-2007,2007,Unprovoked,EGYPT,Daedalus Reef,Swimming,Kristina Aleksandrova,F,18 or 20,"Severe lacerations to lower left leg, ankle an...",N,Oceanic whitetip sharks were in the vicinity,EG,24.934445,35.868120,NaN
1866,2103,24-Jun-2007,2007,Unprovoked,USA,"Silva's Channel, Mokuleia, O'ahu",Surfing,E. Yamada,M,NaN,"No injury, board damaged",N,Tiger shark,US,49.989220,-2.238470,NaN
2203,2496,Jul-2003,2003,Unprovoked,NEW ZEALAND,Beveridge Reef,Snorkeling,Allen --,M,NaN,Chest & buttocks bitten,N,Grey reef shark,NZ,-20.001741,-167.756376,NaN


Get all rows where the fetched and given country codes do not match

In [46]:
df_mismatch = df[
    df["CountryCode"].astype(str).str.upper().replace({"NAN": None}).notna()
    & df["fetched_country_code"].astype(str).str.upper().replace({"NAN": None}).notna()
    & (
        df["CountryCode"].astype(str).str.upper()
        != df["fetched_country_code"].astype(str).str.upper()
    )
].copy()

In [47]:
display(df_mismatch)

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,-35.008658,138.529591,AU
2,2,3rd January,2026,Unprovoked,New Caledonia,Between Bourail and Moindou,Scuba Diving,Unknown,M,?,Injuries to upper limbs,N,Unknown,NC,-21.567724,165.497130,FR
9,9,9th November,2025,Unprovoked,French Polynesia,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,3m shark,PF,30.248169,-85.659363,US
10,10,5th November,2025,Unprovoked,USA,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Unknown,US,36.732382,27.628300,TR
13,13,14th October,2025,Unprovoked,Columbia,Catagena Province,Swimming with sharks,Male child,M,14,Severe hand injury,N,Nurse shark,CA,46.102954,11.129743,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,2792,14-Apr-2000,2000,Unprovoked,USA,"On the south side of Ponce Inlet, Volusia County",Walking,Adam Metz,M,34,Left foot lacerated,N,NaN,US,36.202557,36.164099,TR
2481,2798,15-Mar-2000,2000,Unprovoked,NEW CALEDONIA,Poum,Spearfishing,Gilbert Bui Van Minh,M,35,FATAL,Y,Tiger shark?,NC,-20.231655,164.029894,FR
2482,2799,14-Mar-2000,2000,Unprovoked,AUSTRALIA,"McMasters Beach, Central Coast",Surfing,Craig Ruth,M,NaN,No Injury,N,"Tiger shark, 4 m [13'] ?",AU,44.353110,-74.215714,US
2486,2804,Mar-2000,2000,Unprovoked,USA,Midnight Lump (38 miles offshore),Spearfishing,Kurt Bickel,M,39,"No injury to diver, speargun damaged",N,"Shortfin mako shark, 3 m to 3.4 m [10' to 11']",US,10.552831,119.111687,PH


Fetch new coordinates without clean up first

In [48]:
def get_coordinates(location, countryCode):
    headers = {"User-Agent": "shark-attack-map/1.0 (contact: e12217684@student.tuwien.ac.at)"}
    params = {"q": f"{location} {countryCode}", "format": "json", "limit": 1}
    response = requests.get("https://nominatim.openstreetmap.org/search",headers=headers, params=params)
    if response.status_code == 200 and len(response.json()) != 0:
        print(response.json())
        return (response.json()[0]["lat"], response.json()[0]["lon"])
    else:
        return (None, None)

In [49]:
df_mismatch[["latitude", "longitude"]] = df_mismatch.apply(
    lambda row: get_coordinates(row["Location"], row["CountryCode"]), axis=1, result_type="expand"
)

[{'place_id': 284298843, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 335582694, 'lat': '18.4476715', 'lon': '-65.9994824', 'class': 'natural', 'type': 'beach', 'place_rank': 22, 'importance': 0.10672362325474935, 'addresstype': 'beach', 'name': 'Balneario de Carolina', 'display_name': 'Balneario de Carolina, Cangrejo Arriba, Carolina, Puerto Rico, United States', 'boundingbox': ['18.4436745', '18.4516639', '-66.0080412', '-65.9955523']}]
[{'place_id': 117165, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 2299593704, 'lat': '-8.9263561', 'lon': '-140.1002729', 'class': 'natural', 'type': 'bay', 'place_rank': 22, 'importance': 0.10671028992141601, 'addresstype': 'bay', 'name': 'Baie de Taiohae', 'display_name': 'Baie de Taiohae, Taiohae, Polynésie Française, 98742, France', 'boundingbox': ['-8.9264061', '-8.9263061', '-140.1003229', '-140.1002229']}]
[{

In [50]:
display(df_mismatch)
df_mismatch.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,None,None,AU
2,2,3rd January,2026,Unprovoked,New Caledonia,Between Bourail and Moindou,Scuba Diving,Unknown,M,?,Injuries to upper limbs,N,Unknown,NC,None,None,FR
9,9,9th November,2025,Unprovoked,French Polynesia,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,3m shark,PF,None,None,US
10,10,5th November,2025,Unprovoked,USA,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Unknown,US,None,None,TR
13,13,14th October,2025,Unprovoked,Columbia,Catagena Province,Swimming with sharks,Male child,M,14,Severe hand injury,N,Nurse shark,CA,None,None,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,2792,14-Apr-2000,2000,Unprovoked,USA,"On the south side of Ponce Inlet, Volusia County",Walking,Adam Metz,M,34,Left foot lacerated,N,NaN,US,None,None,TR
2481,2798,15-Mar-2000,2000,Unprovoked,NEW CALEDONIA,Poum,Spearfishing,Gilbert Bui Van Minh,M,35,FATAL,Y,Tiger shark?,NC,-20.2316547,164.0298940,FR
2482,2799,14-Mar-2000,2000,Unprovoked,AUSTRALIA,"McMasters Beach, Central Coast",Surfing,Craig Ruth,M,NaN,No Injury,N,"Tiger shark, 4 m [13'] ?",AU,None,None,US
2486,2804,Mar-2000,2000,Unprovoked,USA,Midnight Lump (38 miles offshore),Spearfishing,Kurt Bickel,M,39,"No injury to diver, speargun damaged",N,"Shortfin mako shark, 3 m to 3.4 m [10' to 11']",US,None,None,PH


Unnamed: 0              285
Date                    285
Year                    285
Type                    285
Country                 285
Location                285
Activity                277
Name                    280
Sex                     272
Age                     219
Injury                  283
Fatal Y/N               280
Species                 187
CountryCode             285
latitude                 75
longitude                75
fetched_country_code    285
dtype: int64

In [51]:
df_mismatch_with_coords = df_mismatch[
    df_mismatch["latitude"].notna() & df_mismatch["longitude"].notna()
].copy()
display(df_mismatch_with_coords)
df_mismatch_with_coords.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code
25,25,1st August,2025,Unprovoked,Puerto Rico,Carolina Beach,Wading,Eleonora Boi,F,39,Bite to thigh area,N,Undetermined,PR,18.4476715,-65.9994824,US
27,27,25th July,2025,Unprovoked,French Polynesia,Taiohae Bay,Swimming,Male child,M,7,Injuries to right forearm left hand and calf w...,N,Undetermined,PF,-8.9263561,-140.1002729,FR
78,83,2024-09-17 00:00:00,2024,Unprovoked,Honduras,Puerto Castillo,Diving for shellfish,Fernando Mendoza Ocampo,M,33,Bite to left leg,N,Bull shark 6-7ft,HN,28.1357805,-15.4292537,ES
126,134,21 Nov-2023,2023,Unprovoked,BAHAMAS,Tiger Beach,Scuba diving,female,F,47,FATAL,Y,NaN,BS,56.4966701,-2.7348713,VN
204,220,19-Feb-2023,2023,Unprovoked,NEW CALEDONIA,"Château-Royal Beach , Noumea",Swimming,Chris Davis,M,59,"FATAL, Multiple injuries",F,Bull shark,NC,-22.3098310,166.4484302,FR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,2495,Late Jul-2003,2003,Unprovoked,FRENCH POLYNESIA,Moorea,Spearfishing,male,M,NaN,Leg bitten,N,NaN,PF,-17.5333060,-149.8386006,FR
2251,2548,Nov-2002,2002,Unprovoked,FRENCH POLYNESIA,Tetiaroa,"Fishing, standing in 2' of water",Scott Heywood,M,52,Shin bruised,N,NaN,PF,-17.0162552,-149.5897766,FR
2472,2788,13-May-2000,2000,Sea Disaster,NEW CALEDONIA,Mont Dore,Air Disaster - Piper aircraft crashed into the...,3 people,NaN,NaN,Sharks prevented recovery of remains,Y,Tiger sharks & bull sharks (20 sharks in all),NC,-22.2265112,166.5255445,FR
2481,2798,15-Mar-2000,2000,Unprovoked,NEW CALEDONIA,Poum,Spearfishing,Gilbert Bui Van Minh,M,35,FATAL,Y,Tiger shark?,NC,-20.2316547,164.0298940,FR


Unnamed: 0              75
Date                    75
Year                    75
Type                    75
Country                 75
Location                75
Activity                72
Name                    74
Sex                     71
Age                     58
Injury                  74
Fatal Y/N               73
Species                 44
CountryCode             75
latitude                75
longitude               75
fetched_country_code    75
dtype: int64

In [53]:
coord_notfound = df_mismatch.loc[df_mismatch["latitude"].isna()].copy()
display(coord_notfound)

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,None,None,AU
2,2,3rd January,2026,Unprovoked,New Caledonia,Between Bourail and Moindou,Scuba Diving,Unknown,M,?,Injuries to upper limbs,N,Unknown,NC,None,None,FR
9,9,9th November,2025,Unprovoked,French Polynesia,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,3m shark,PF,None,None,US
10,10,5th November,2025,Unprovoked,USA,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Unknown,US,None,None,TR
13,13,14th October,2025,Unprovoked,Columbia,Catagena Province,Swimming with sharks,Male child,M,14,Severe hand injury,N,Nurse shark,CA,None,None,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,2769,12-Jul-2000,2000,Unprovoked,TONGA,Treated at Nuku-alofa,Scuba diving,Christian Eckoff,M,69,Left arm bitten,N,"Grey reef shark, 2 m [6.75']",TO,None,None,AT
2471,2787,Early Jun-2000,2000,Unprovoked,TANZANIA,"Coco Beach in Oyster Bay, 7 km north of Dar-es...",Swimming,male,M,NaN,"FATAL, legs severed",Y,Thought to involve a Zambesi shark,TZ,None,None,PH
2475,2792,14-Apr-2000,2000,Unprovoked,USA,"On the south side of Ponce Inlet, Volusia County",Walking,Adam Metz,M,34,Left foot lacerated,N,NaN,US,None,None,TR
2482,2799,14-Mar-2000,2000,Unprovoked,AUSTRALIA,"McMasters Beach, Central Coast",Surfing,Craig Ruth,M,NaN,No Injury,N,"Tiger shark, 4 m [13'] ?",AU,None,None,US


Cleanup with same steps as before

In [55]:
# Regex-Bausteine
RE_PARENS = re.compile(r"\([^)]*\)")
RE_BETWEEN = re.compile(r"\bbetween\s+(?P<a>.+?)\s+(?:and|&)\s+(?P<b>.+)\b", re.IGNORECASE)
RE_CHANNEL_BETWEEN = re.compile(r"\bchannel\s+between\s+(?P<a>.+?)\s+(?:and|&)\s+(?P<b>.+)\b", re.IGNORECASE)
RE_OFF = re.compile(r"\b(?:\d+\s*(?:nm|km|miles?)\s+)?off\s+(?P<place>.+)\b", re.IGNORECASE)

# Distanzen/“about 500 km (310 miles)” usw.
RE_DISTANCE = re.compile(r"\babout\s+\d+.*?$|\b\d+\s*(?:km|m|nm|miles?)\b", re.IGNORECASE)

# Richtungsausdrücke (macht’s simpler)
RE_DIRECTION_OF = re.compile(r"\b(north|south|east|west)\s+of\s+", re.IGNORECASE)

def simplify_location(loc: str) -> str:
    if not isinstance(loc, str) or not loc.strip():
        return ""

    s = loc.strip()

    # 1) Klammern entfernen
    s = RE_PARENS.sub("", s)

    # 2) komische Quotes normalisieren/entfernen
    s = s.replace("“", '"').replace("”", '"').replace("’", "'").replace("`", "'")

    # 3) "between A and B" => A
    m = RE_CHANNEL_BETWEEN.search(s)
    if m:
        s = m.group("a")
    else:
        m = RE_BETWEEN.search(s)
        if m:
            s = m.group("a")

    # 4) "off X" => X (auch "70 miles off Pensacola")
    m = RE_OFF.search(s)
    if m:
        s = m.group("place")

    # 5) "north of X" => X (wir wollen was Geocodable)
    s = RE_DIRECTION_OF.sub("", s)

    # 6) "near Y" => Y (nimmt den Teil nach dem letzten 'near')
    if re.search(r"\bnear\b", s, flags=re.IGNORECASE):
        s = re.split(r"\bnear\b", s, flags=re.IGNORECASE)[-1]

    # 7) Distanzen/“about …” raus
    s = RE_DISTANCE.sub("", s)

    # 8) Fragezeichen/seltsame Satzzeichen raus
    s = s.replace("?", "").replace("!", "").replace(";", "").replace("/", "")
    s = s.replace('"', "").replace("'", "")  # Apostrophe/Quotes komplett raus (einfachster Weg)

    # 9) Mehrfachspaces/Kommas normalisieren
    s = re.sub(r"\s+", " ", s).strip()
    s = s.strip(" ,.-")

    # 10) optional: nur erster Teil vor Komma (sehr “einfacher” fallback)
    #    (wenn du lieber mehr Kontext behalten willst, kommentier die nächste Zeile aus)
    if "," in s:
        s = s.split(",")[0].strip()

    return s


In [56]:
coord_notfound["cleaned_location"] = coord_notfound["Location"].apply(simplify_location)

In [58]:
coord_notfound[["latitude", "longitude"]] = coord_notfound.apply(
    lambda row: get_coordinates(row["cleaned_location"], row["CountryCode"]), axis=1, result_type="expand"
)

[{'place_id': 23187605, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 378262, 'lat': '-21.5677243', 'lon': '165.4971296', 'class': 'boundary', 'type': 'administrative', 'place_rank': 16, 'importance': 0.18671551155870905, 'addresstype': 'village', 'name': 'Bourail', 'display_name': 'Bourail, Province Sud, Nouvelle-Calédonie, 98870, France', 'boundingbox': ['-21.6687902', '-21.3698221', '165.2310061', '165.6204768']}]
[{'place_id': 22965945, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 378435, 'lat': '-20.5613596', 'lon': '164.2811715', 'class': 'boundary', 'type': 'administrative', 'place_rank': 16, 'importance': 0.40554112705548934, 'addresstype': 'town', 'name': 'Koumac', 'display_name': 'Koumac, Province Nord, Nouvelle-Calédonie, 98850, France', 'boundingbox': ['-20.6340756', '-20.3696556', '164.1166474', '164.5524509']}]
[{'place_id': 1516

In [59]:
display(coord_notfound)
coord_notfound.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code,cleaned_location
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,None,None,AU,Dorsch Beach
2,2,3rd January,2026,Unprovoked,New Caledonia,Between Bourail and Moindou,Scuba Diving,Unknown,M,?,Injuries to upper limbs,N,Unknown,NC,-21.5677243,165.4971296,FR,Bourail
9,9,9th November,2025,Unprovoked,French Polynesia,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,3m shark,PF,None,None,US,Hakahau Bay
10,10,5th November,2025,Unprovoked,USA,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Unknown,US,None,None,TR,Pine Trees Hanalei Bay Kaui
13,13,14th October,2025,Unprovoked,Columbia,Catagena Province,Swimming with sharks,Male child,M,14,Severe hand injury,N,Nurse shark,CA,None,None,IT,Catagena Province
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,2769,12-Jul-2000,2000,Unprovoked,TONGA,Treated at Nuku-alofa,Scuba diving,Christian Eckoff,M,69,Left arm bitten,N,"Grey reef shark, 2 m [6.75']",TO,None,None,AT,Treated at Nuku-alofa
2471,2787,Early Jun-2000,2000,Unprovoked,TANZANIA,"Coco Beach in Oyster Bay, 7 km north of Dar-es...",Swimming,male,M,NaN,"FATAL, legs severed",Y,Thought to involve a Zambesi shark,TZ,None,None,PH,Coco Beach in Oyster Bay
2475,2792,14-Apr-2000,2000,Unprovoked,USA,"On the south side of Ponce Inlet, Volusia County",Walking,Adam Metz,M,34,Left foot lacerated,N,NaN,US,None,None,TR,On the south side of Ponce Inlet
2482,2799,14-Mar-2000,2000,Unprovoked,AUSTRALIA,"McMasters Beach, Central Coast",Surfing,Craig Ruth,M,NaN,No Injury,N,"Tiger shark, 4 m [13'] ?",AU,None,None,US,McMasters Beach


Unnamed: 0              210
Date                    210
Year                    210
Type                    210
Country                 210
Location                210
Activity                205
Name                    206
Sex                     201
Age                     161
Injury                  209
Fatal Y/N               207
Species                 143
CountryCode             210
latitude                 24
longitude                24
fetched_country_code    210
cleaned_location        210
dtype: int64

In [60]:
coord_notfound_2 = coord_notfound.loc[coord_notfound["latitude"].isna()].copy()
display(coord_notfound_2)

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code,cleaned_location
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,None,None,AU,Dorsch Beach
9,9,9th November,2025,Unprovoked,French Polynesia,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,3m shark,PF,None,None,US,Hakahau Bay
10,10,5th November,2025,Unprovoked,USA,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Unknown,US,None,None,TR,Pine Trees Hanalei Bay Kaui
13,13,14th October,2025,Unprovoked,Columbia,Catagena Province,Swimming with sharks,Male child,M,14,Severe hand injury,N,Nurse shark,CA,None,None,IT,Catagena Province
19,19,1st September,2025,Unprovoked,USA,Horseshoe reef Key Largo,Snorkeling,Richard Burrows,M,8,Bite to leg,N,Not stated,US,None,None,EG,Horseshoe reef Key Largo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,2769,12-Jul-2000,2000,Unprovoked,TONGA,Treated at Nuku-alofa,Scuba diving,Christian Eckoff,M,69,Left arm bitten,N,"Grey reef shark, 2 m [6.75']",TO,None,None,AT,Treated at Nuku-alofa
2471,2787,Early Jun-2000,2000,Unprovoked,TANZANIA,"Coco Beach in Oyster Bay, 7 km north of Dar-es...",Swimming,male,M,NaN,"FATAL, legs severed",Y,Thought to involve a Zambesi shark,TZ,None,None,PH,Coco Beach in Oyster Bay
2475,2792,14-Apr-2000,2000,Unprovoked,USA,"On the south side of Ponce Inlet, Volusia County",Walking,Adam Metz,M,34,Left foot lacerated,N,NaN,US,None,None,TR,On the south side of Ponce Inlet
2482,2799,14-Mar-2000,2000,Unprovoked,AUSTRALIA,"McMasters Beach, Central Coast",Surfing,Craig Ruth,M,NaN,No Injury,N,"Tiger shark, 4 m [13'] ?",AU,None,None,US,McMasters Beach


In [67]:
def strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

def generate_queries(loc: str, country_name: str | None = None):
    if not isinstance(loc, str) or not loc.strip():
        return []
    s = loc.strip()

    # Normalisieren
    s = strip_accents(s)
    s = s.replace("-", " ")
    s = s.replace("’", "'").replace("`", "'").replace('"', "")
    s = re.sub(r"\s+", " ", s).strip()

    variants = []

    # 1) Original
    variants.append(s)

    # 2) Apostrophe komplett weg (einfachster “works often”)
    variants.append(s.replace("'", ""))

    # 3) Häufige Tippfehler / Varianten
    variants.append(re.sub(r"\bbaay\b", "bay", s, flags=re.IGNORECASE))  # Crowdy Baay -> Crowdy Bay
    variants.append(re.sub(r"\bbeach\b", "bay", s, flags=re.IGNORECASE)) # beach -> bay
    variants.append(re.sub(r"\bbeach\b", "cove", s, flags=re.IGNORECASE))
    variants.append(re.sub(r"\bbeach\b", "", s, flags=re.IGNORECASE).strip())  # beach einfach entfernen

    # 4) Wenn mehrere Wörter: nur “Kern” nehmen (z.B. "Prevelly Beach Magaret River" -> "Prevelly Beach")
    tokens = s.split()
    if len(tokens) >= 3:
        variants.append(" ".join(tokens[:2]))
        variants.append(" ".join(tokens[:3]))

    # 5) Stop-Wörter abschneiden (sehr pragmatisch)
    for stop in [" river", " island", " atoll", " point", " reef", " jetty", " harbour", " harbor"]:
        if stop in s.lower():
            cut = s.lower().split(stop)[0].strip()
            if cut:
                variants.append(cut)

    # Dedupe + leere raus
    out = []
    seen = set()
    for v in variants:
        v = (v or "").strip(" ,.")
        if len(v) < 2:
            continue
        key = v.lower()
        if key not in seen:
            out.append(v)
            seen.add(key)

    if country_name and isinstance(country_name, str) and country_name.strip():
        out = [f"{q}, {country_name.strip()}" for q in out] + out

    extra = []

    for v in out:
        v2 = remove_directional_phrases(v)
        if v2 != v:
            extra.append(v2)

        extra.append(keep_last_place_chunk(v))
        extra.append(remove_double_letters(v))

        extra.extend(beach_synonym_variants(v))
        extra.append(last_token_only(v))

    # dedupe
    for v in extra:
        v = v.strip(" ,.")
        if len(v) >= 2 and v.lower() not in seen:
            out.append(v)
            seen.add(v.lower())    

    return out


In [68]:
def get_coordinates_best_effort(cleaned_location, country_code):
    headers = {"User-Agent": "shark-attack-map/1.0 (contact: e12217684@student.tuwien.ac.at)"}
    

    for q in generate_queries(cleaned_location):

        params = {
            "q": f"{q} {country_code}",
            "format": "json",
            "limit": 1,
        }
        res = requests.get("https://nominatim.openstreetmap.org/search", headers=headers, params=params)
        if res.status_code == 200:
            data = res.json()
            if data:
                print(data)
                latlon = (float(data[0]["lat"]), float(data[0]["lon"]))
                return latlon
    return (None, None)

In [64]:
coord_notfound_2[["latitude", "longitude"]] = coord_notfound_2.apply(
    lambda row: get_coordinates_best_effort(row["Location"], row["CountryCode"]),
    axis=1,
    result_type="expand"
)


[{'place_id': 17603162, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 38699948, 'lat': '-35.0086582', 'lon': '138.5295913', 'class': 'highway', 'type': 'residential', 'place_rank': 26, 'importance': 0.05340822384985119, 'addresstype': 'road', 'name': 'Dorsch Street', 'display_name': 'Dorsch Street, Hove, Adelaide, City of Holdfast Bay, South Australia, 5048, Australia', 'boundingbox': ['-35.0095907', '-35.0077256', '138.5295169', '138.5296658']}]
[{'place_id': 297057967, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 4272451854, 'lat': '34.0879551', 'lon': '-118.2915983', 'class': 'natural', 'type': 'tree', 'place_rank': 30, 'importance': 8.738155184248139e-05, 'addresstype': 'natural', 'name': 'Pine', 'display_name': 'Pine, North Vermont Avenue, East Hollywood, Los Angeles, Los Angeles County, California, 90029, United States', 'boundingbox': ['34.08790

In [65]:
display(coord_notfound_2)
coord_notfound_2.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code,cleaned_location
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,-35.008658,138.529591,AU,Dorsch Beach
9,9,9th November,2025,Unprovoked,French Polynesia,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,3m shark,PF,NaN,NaN,US,Hakahau Bay
10,10,5th November,2025,Unprovoked,USA,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Unknown,US,34.087955,-118.291598,TR,Pine Trees Hanalei Bay Kaui
13,13,14th October,2025,Unprovoked,Columbia,Catagena Province,Swimming with sharks,Male child,M,14,Severe hand injury,N,Nurse shark,CA,NaN,NaN,IT,Catagena Province
19,19,1st September,2025,Unprovoked,USA,Horseshoe reef Key Largo,Snorkeling,Richard Burrows,M,8,Bite to leg,N,Not stated,US,42.877758,-78.889747,EG,Horseshoe reef Key Largo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,2769,12-Jul-2000,2000,Unprovoked,TONGA,Treated at Nuku-alofa,Scuba diving,Christian Eckoff,M,69,Left arm bitten,N,"Grey reef shark, 2 m [6.75']",TO,57.128827,-2.180670,AT,Treated at Nuku-alofa
2471,2787,Early Jun-2000,2000,Unprovoked,TANZANIA,"Coco Beach in Oyster Bay, 7 km north of Dar-es...",Swimming,male,M,NaN,"FATAL, legs severed",Y,Thought to involve a Zambesi shark,TZ,-6.769082,39.282187,PH,Coco Beach in Oyster Bay
2475,2792,14-Apr-2000,2000,Unprovoked,USA,"On the south side of Ponce Inlet, Volusia County",Walking,Adam Metz,M,34,Left foot lacerated,N,NaN,US,27.814874,-97.388711,TR,On the south side of Ponce Inlet
2482,2799,14-Mar-2000,2000,Unprovoked,AUSTRALIA,"McMasters Beach, Central Coast",Surfing,Craig Ruth,M,NaN,No Injury,N,"Tiger shark, 4 m [13'] ?",AU,-33.500570,151.327410,US,McMasters Beach


Unnamed: 0              186
Date                    186
Year                    186
Type                    186
Country                 186
Location                186
Activity                182
Name                    182
Sex                     178
Age                     144
Injury                  185
Fatal Y/N               183
Species                 125
CountryCode             186
latitude                 92
longitude                92
fetched_country_code    186
cleaned_location        186
dtype: int64

In [66]:
coord_notfound_3 = coord_notfound_2.loc[coord_notfound_2["latitude"].isna()].copy()

In [69]:
def remove_directional_phrases(s: str) -> str:
    patterns = [
        r"\bnorth of\b", r"\bsouth of\b", r"\beast of\b", r"\bwest of\b",
        r"\bnear\b", r"\boff\b", r"\boff the coast of\b",
        r"\bat the mouth of\b", r"\bin the vicinity of\b",
        r"\bin the area of\b", r"\bin front of\b"
    ]
    out = s
    for p in patterns:
        out = re.sub(p, "", out, flags=re.IGNORECASE)
    return re.sub(r"\s+", " ", out).strip()

def keep_last_place_chunk(s: str) -> str:
    # trennt bei typischen Präpositionen und nimmt das letzte Segment
    for sep in [" near ", " off ", " at ", " in ", " on "]:
        if sep in s.lower():
            return s.lower().split(sep)[-1].strip()
    return s

def beach_synonym_variants(s: str) -> list[str]:
    variants = []
    if re.search(r"\bbeach\b", s, flags=re.IGNORECASE):
        variants.extend([
            re.sub(r"\bbeach\b", "bay", s, flags=re.IGNORECASE),
            re.sub(r"\bbeach\b", "cove", s, flags=re.IGNORECASE),
            re.sub(r"\bbeach\b", "point", s, flags=re.IGNORECASE),
            re.sub(r"\bbeach\b", "", s, flags=re.IGNORECASE).strip(),
        ])
    return variants

def last_token_only(s: str) -> str:
    toks = s.split()
    if len(toks) >= 2:
        return toks[-1]
    return s

def remove_double_letters(s: str) -> str:
    # z.B. baay -> bay, coove -> cove
    return re.sub(r"(.)\1+", r"\1", s)

In [70]:
coord_notfound_3[["latitude", "longitude"]] = coord_notfound_3.apply(
    lambda row: get_coordinates_best_effort(row["Location"], row["CountryCode"]),
    axis=1,
    result_type="expand"
)


[{'place_id': 333976430, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 5119438721, 'lat': '42.3521152', 'lon': '-71.0672396', 'class': 'amenity', 'type': 'restaurant', 'place_rank': 30, 'importance': 8.102616922250732e-05, 'addresstype': 'amenity', 'name': "P.F. Chang's", 'display_name': "P.F. Chang's, 8, Park Plaza, Bay Village, Back Bay, Boston, Suffolk County, Massachusetts, 02116, United States", 'boundingbox': ['42.3520652', '42.3521652', '-71.0672896', '-71.0671896']}]
[{'place_id': 349015603, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 391186, 'lat': '55.0012510', 'lon': '-115.0021360', 'class': 'boundary', 'type': 'administrative', 'place_rank': 8, 'importance': 0.7165693323280248, 'addresstype': 'state', 'name': 'Alberta', 'display_name': 'Alberta, Canada', 'boundingbox': ['48.9966650', '60.0004699', '-120.0013835', '-110.0047639']}]
[{'

In [71]:
display(coord_notfound_3)
coord_notfound_3.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code,cleaned_location
9,9,9th November,2025,Unprovoked,French Polynesia,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,3m shark,PF,42.352115,-71.067240,US,Hakahau Bay
13,13,14th October,2025,Unprovoked,Columbia,Catagena Province,Swimming with sharks,Male child,M,14,Severe hand injury,N,Nurse shark,CA,55.001251,-115.002136,IT,Catagena Province
43,43,2025-05-26 00:00:00,2025,Unprovoked,Vanuatu,Espiitu Santo Island,Swimming,Tumas,M,14,Multiple injuries to body one hand and leg bit...,Y,Not stated,VU,48.877187,2.333572,IS,Espiitu Santo Island
73,77,2024-11-01 00:00:00,2024,Unprovoked,Hawaii,Sand Piles Waiehu Beach Park,Surfing,Kenji,M,61,Right leg severted below knee,N,Not stated,US,39.108930,-105.756164,MM,Sand Piles Waiehu Beach Park
105,110,2024-04-20 00:00:00,2024,Unprovoked,Australia,Lighthouse bombie surf spot Exemouth,Surfing,Augustin Sanchez,M,26,Minor injuries left foot,N,Not specified,AU,35.993102,-83.860618,US,Lighthouse bombie surf spot Exemouth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,2583,Reported 13-Jun-2002,2002,Unprovoked,PAPUA NEW GUINEA,"Brooker Island , Calvados Chain",NaN,NaN,NaN,NaN,"Arm severely lacerated, surgically amputated",N,NaN,PG,-11.166667,152.750000,US,Brooker Island
2297,2595,21-May-2002,2002,Unprovoked,PAPUA NEW GUINEA,Tewatewa Island,Collecting beche-de-mer,Billy Leonard,M,14,Wrist lacerated,N,NaN,PG,44.496613,14.946185,IS,Tewatewa Island
2404,2716,24-Dec-2000,2000,Unprovoked,AUSTRALIA,Flinders Cay,Scuba diving,male,M,23,Hand bitten,N,NaN,AU,-9.141863,143.875643,FR,Flinders Cay
2441,2755,21-Aug-2000,2000,Unprovoked,USA,"Bouges Bank, Emerald Isle, Carteret County",Swimming out to porpoises,male,M,NaN,"Severe gash to left hand above wrist, almost s...",N,NaN,US,48.212941,-106.604884,GB,Bouges Bank


Unnamed: 0              94
Date                    94
Year                    94
Type                    94
Country                 94
Location                94
Activity                91
Name                    91
Sex                     88
Age                     68
Injury                  93
Fatal Y/N               91
Species                 65
CountryCode             94
latitude                88
longitude               88
fetched_country_code    94
cleaned_location        94
dtype: int64

In [73]:
coord_notfound_4 = coord_notfound_3.loc[coord_notfound_3["latitude"].isna()].copy()
display(coord_notfound_4)

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code,cleaned_location
106,111,2024-04-08 00:00:00,2024,Unprovoked,Maldive Islands,Unspecified,Swimming,Not specified,M,?,Minor injury to left leg,N,Not specified,MV,NaN,NaN,PK,Unspecified
176,188,28-May 2023,2023,Unprovoked,NEW CALEDONIA,Twagne,Spearfishing,male,M,42,"Fatal, bite to leg, shoulder and head",Y,NaN,NC,NaN,NaN,FR,Twagne
947,1059,19-Dec-2015,2015,Unprovoked,ARUBA,Boat capsized,Sea disaster,Adrian Esteban Rafael,M,58,FATAL,Y,NaN,AW,NaN,NaN,US,Boat capsized
1134,1271,Reported 27-Jun-2014,2014,Watercraft,ST. MARTIN,20 miles from shore,Transatlantic Rowing,Victor Mooney,M,48,His boat was holed by a shark,N,Oceanic whitetip shark',MF,NaN,NaN,US,from shore
1471,1656,21-May-2011,2011,Unprovoked,NEW CALEDONIA,Kendec,Kite Boarding,Nathan ____,M,15,"Thigh bitten, FATAL",Y,"Tiger shark, 2.8m",NC,NaN,NaN,FR,Kendec
1885,2127,22-Mar-2007,2007,Unprovoked,YEMEN,Ras-Alkalb,Murder,At least 29 (and possibly another 71) Somali &...,NaN,NaN,"FATAL, beaten & thrown overboard by smugglers...",Y,NaN,YE,NaN,NaN,MR,Ras-Alkalb


In [74]:
def update_coords(target_df, source_df):
    mask = source_df["latitude"].notna() & source_df["longitude"].notna()
    target_df.loc[source_df.index[mask], ["latitude", "longitude"]] = \
        source_df.loc[mask, ["latitude", "longitude"]]

In [75]:
update_coords(df_mismatch, coord_notfound)
update_coords(df_mismatch, coord_notfound_2)
update_coords(df_mismatch, coord_notfound_3)
update_coords(df_mismatch, coord_notfound_4)

In [77]:
display(df_mismatch)
df_mismatch.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,-35.008658,138.529591,AU
2,2,3rd January,2026,Unprovoked,New Caledonia,Between Bourail and Moindou,Scuba Diving,Unknown,M,?,Injuries to upper limbs,N,Unknown,NC,-21.5677243,165.4971296,FR
9,9,9th November,2025,Unprovoked,French Polynesia,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,3m shark,PF,42.352115,-71.06724,US
10,10,5th November,2025,Unprovoked,USA,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Unknown,US,34.087955,-118.291598,TR
13,13,14th October,2025,Unprovoked,Columbia,Catagena Province,Swimming with sharks,Male child,M,14,Severe hand injury,N,Nurse shark,CA,55.001251,-115.002136,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,2792,14-Apr-2000,2000,Unprovoked,USA,"On the south side of Ponce Inlet, Volusia County",Walking,Adam Metz,M,34,Left foot lacerated,N,NaN,US,27.814874,-97.388711,TR
2481,2798,15-Mar-2000,2000,Unprovoked,NEW CALEDONIA,Poum,Spearfishing,Gilbert Bui Van Minh,M,35,FATAL,Y,Tiger shark?,NC,-20.2316547,164.0298940,FR
2482,2799,14-Mar-2000,2000,Unprovoked,AUSTRALIA,"McMasters Beach, Central Coast",Surfing,Craig Ruth,M,NaN,No Injury,N,"Tiger shark, 4 m [13'] ?",AU,-33.50057,151.32741,US
2486,2804,Mar-2000,2000,Unprovoked,USA,Midnight Lump (38 miles offshore),Spearfishing,Kurt Bickel,M,39,"No injury to diver, speargun damaged",N,"Shortfin mako shark, 3 m to 3.4 m [10' to 11']",US,33.574002,-117.825827,PH


Unnamed: 0              285
Date                    285
Year                    285
Type                    285
Country                 285
Location                285
Activity                277
Name                    280
Sex                     272
Age                     219
Injury                  283
Fatal Y/N               280
Species                 187
CountryCode             285
latitude                279
longitude               279
fetched_country_code    285
dtype: int64

In [78]:
df_mismatch.to_csv("/Users/emilywu/Desktop/DataLiteracy/MiniProjekt/mismatch_six_missing.csv")

Manually adding missing coordinates

In [85]:
manual_coords = {
    111:  (4.175463, 73.508804),        # Maldives (Unspecified) -> Malé (Capital) :contentReference[oaicite:0]{index=0}
    188:  (-20.2378690, 164.0290489),   # Twagne, Poum (NC) -> Poum area (approx) :contentReference[oaicite:1]{index=1}
    1059: (12.510052, -70.009354),      # Aruba (Boat capsized) -> Oranjestad / near Aruba (approx) :contentReference[oaicite:2]{index=2}
    1271: (18.35, -63.06),              # St. Martin "20 miles from shore" -> offshore north of St Martin (approx) :contentReference[oaicite:3]{index=3}
    1656: (-20.666667, 164.25),         # Île Kendec (NC) :contentReference[oaicite:4]{index=4}
    2127: (14.044439, 48.667783),       # Ras-Alkalb, Yemen :contentReference[oaicite:5]{index=5}
}

In [90]:
for idx, (lat, lon) in manual_coords.items():
    df_mismatch.loc[idx, ["latitude", "longitude"]] = lat, lon

In [91]:
display(df_mismatch)
df_mismatch.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude,fetched_country_code
1,1.0,8th January,2026.0,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,-35.008658,138.529591,AU
2,2.0,3rd January,2026.0,Unprovoked,New Caledonia,Between Bourail and Moindou,Scuba Diving,Unknown,M,?,Injuries to upper limbs,N,Unknown,NC,-21.5677243,165.4971296,FR
9,9.0,9th November,2025.0,Unprovoked,French Polynesia,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,3m shark,PF,42.352115,-71.06724,US
10,10.0,5th November,2025.0,Unprovoked,USA,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Unknown,US,34.087955,-118.291598,TR
13,13.0,14th October,2025.0,Unprovoked,Columbia,Catagena Province,Swimming with sharks,Male child,M,14,Severe hand injury,N,Nurse shark,CA,55.001251,-115.002136,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.175463,73.508804,NaN
1059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.510052,-70.009354,NaN
1271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.35,-63.06,NaN
1656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-20.666667,164.25,NaN


Unnamed: 0              285
Date                    285
Year                    285
Type                    285
Country                 285
Location                285
Activity                277
Name                    280
Sex                     272
Age                     219
Injury                  283
Fatal Y/N               280
Species                 187
CountryCode             285
latitude                284
longitude               284
fetched_country_code    285
dtype: int64

Combine old and new data

In [94]:
clean = pd.read_csv("/Users/emilywu/Desktop/DataLiteracy/MiniProjekt/clean_shark_attacks.csv")
mismatch = pd.read_csv("/Users/emilywu/Desktop/DataLiteracy/MiniProjekt/mismatch_six_missing.csv")

# Key-Spalte festlegen (meist "Unnamed: 0")
KEY = "Unnamed: 0"

# Nur die Spalten nehmen, die du updaten willst (und nur wenn lat/lon wirklich vorhanden sind)
m_upd = mismatch[[KEY, "latitude", "longitude"]].copy()
m_upd["latitude"] = pd.to_numeric(m_upd["latitude"], errors="coerce")
m_upd["longitude"] = pd.to_numeric(m_upd["longitude"], errors="coerce")
m_upd = m_upd.dropna(subset=["latitude", "longitude"])  # nur echte Updates

# Merge: lat/lon aus mismatch als *_new
clean2 = clean.merge(m_upd, on=KEY, how="left", suffixes=("", "_new"))

# Überschreiben: wenn *_new da ist, nimm das, sonst das alte
clean2["latitude"] = clean2["latitude_new"].combine_first(clean2["latitude"])
clean2["longitude"] = clean2["longitude_new"].combine_first(clean2["longitude"])

clean2 = clean2.drop(columns=["latitude_new", "longitude_new"])

In [95]:
display(clean2)
clean2.count()

,Unnamed: 0,Date,Year,Type,Country,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Species,CountryCode,latitude,longitude
0,0,10th January,2026,Unprovoked,Australia,Avalon Beach,Surfing,Paul Stanton,M,?,puncture mark to left thumb,N,Unknown,AU,-33.636504,151.329030
1,1,8th January,2026,Unprovoked,US Virgin Islands,Dorsch Beach,Snorkeling,Arlene Lillis,F,56,Left arm torn off in the attack below the elbow,Y,Unknown,VI,-35.008658,138.529591
2,2,3rd January,2026,Unprovoked,New Caledonia,Between Bourail and Moindou,Scuba Diving,Unknown,M,?,Injuries to upper limbs,N,Unknown,NC,-21.567724,165.497130
3,3,21st December,2025,Unprovoked,USA,Lovers Point Pacific Grove,Swimming,Erica Fox,F,55,Taken by shark body recovered with multiple in...,Y,Great White Shark,US,36.623394,-121.906667
4,4,12th December,2025,Unprovoked,USA,Salmon Creek,Surfing,Unknown,M,?,Hand Injury,N,Suspected Great White Shark,US,45.712902,-122.664348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2490,2808,03-Feb-2000,2000,Unprovoked,NEW ZEALAND,Oreti Beach (reported as the 4th person bitten...,Surfing,Michael Petas,M,12,"No injury, wetsuit punctured",N,NaN,NZ,-46.421301,168.219569
2491,2809,01-Feb-2000,2000,Unprovoked,AUSTRALIA,"Point Sinclair, Cactus Beach near Penong",Surfing,Anthony Hayes,M,26,Hand bitten,N,3 m [10'] shark,AU,-31.928309,133.011075
2492,2810,Reported 28-Jan-2000,2000,Watercraft,REUNION,Saint Pierre,Canoe with 3 men onboard sank,Boulabhaï Ishmael,M,NaN,FATAL,Y,NaN,RE,-21.341298,55.477617
2493,2811,05-Jan-2000,2000,Unprovoked,THAILAND,Phang nga Island,Diving,Stephan Kahl,M,35,FATAL,Y,NaN,TH,9.334030,98.323241


Unnamed: 0     2495
Date           2495
Year           2495
Type           2495
Country        2495
Location       2495
Activity       2413
Name           2467
Sex            2400
Age            1942
Injury         2488
Fatal Y/N      2482
Species        1586
CountryCode    2495
latitude       2495
longitude      2495
dtype: int64

In [96]:
clean2.to_csv("/Users/emilywu/Desktop/DataLiteracy/MiniProjekt/clean_shark_attacks_updated.csv")

# trying out purposes

In [29]:
res = get_country_code(-21.567724, 165.497130)

In [32]:
print(json.dumps(res, indent=4))
print(res["address"]["country_code"])

{
    "place_id": 23202877,
    "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
    "osm_type": "way",
    "osm_id": 72050112,
    "lat": "-21.5675758",
    "lon": "165.4970244",
    "class": "highway",
    "type": "residential",
    "place_rank": 26,
    "importance": 0.05338217822537572,
    "addresstype": "road",
    "name": "Rue Lacroix",
    "display_name": "Rue Lacroix, Bourail, Province Sud, Nouvelle-Cal\u00e9donie, 98870, France",
    "address": {
        "road": "Rue Lacroix",
        "village": "Bourail",
        "county": "Province Sud",
        "archipelago": "Nouvelle-Cal\u00e9donie",
        "ISO3166-2-lvl3": "FR-NC",
        "postcode": "98870",
        "country": "France",
        "country_code": "fr"
    },
    "boundingbox": [
        "-21.5682539",
        "-21.5673634",
        "165.4960728",
        "165.4973225"
    ]
}
fr


In [ ]:
def get_coordinates(location, countryCode):
    headers = {"User-Agent": "shark-attack-map/1.0 (contact: e12217684@student.tuwien.ac.at)"}
    params = {"q": f"{location}", "format": "json", "countrycodes": f"{countryCode}", "limit": 1}
    response = requests.get("https://nominatim.openstreetmap.org/search",headers=headers, params=params)
    if response.status_code == 200 and len(response.json()) != 0:
        print(response.json())
        return (response.json()[0]["lat"], response.json()[0]["lon"])
    else:
        return (None, None)

In [28]:
headers = {"User-Agent": "shark-attack-map/1.0 (contact: e12217684@student.tuwien.ac.at)"}
params = {"q": "saint pierre reunion", "format": "json", "limit": 1}
response = requests.get("https://nominatim.openstreetmap.org/search",headers=headers, params=params)
print(json.dumps(response.json(), indent=4))

[
    {
        "place_id": 31118988,
        "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
        "osm_type": "relation",
        "osm_id": 315484,
        "lat": "-21.3412980",
        "lon": "55.4776174",
        "class": "boundary",
        "type": "administrative",
        "place_rank": 15,
        "importance": 0.48796654768318326,
        "addresstype": "town",
        "name": "Saint-Pierre",
        "display_name": "Saint-Pierre, La R\u00e9union, France",
        "boundingbox": [
            "-21.3635496",
            "-21.2543907",
            "55.4038017",
            "55.6048057"
        ]
    }
]
